## 3.8 Теоретические вопросы: Свёрточные нейросети в обработке текста

### Импорт необходимых библиотек

In [293]:
import numpy as np

### [Напишите функцию, применяющую свёрточный модуль к последовательности.](https://stepik.org/lesson/261477/step/2?unit=242226)

In [442]:
def apply_convolution(data, kernel, bias):
    """
    data - InLen x InChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutLen x OutChannels
    """

    result = []
    cont = []
    kernel_size = kernel.shape[-1]

    for k_out in kernel:
        for offset in range(len(data)):
            if offset + (kernel_size - 1) < len(data):
                cont.append(
                    (data[offset : offset + kernel_size] * k_out.T).ravel().sum()
                )

        result.append(cont)
        cont = []

    result = np.array(result).T + bias

    return result

### [Напишите функцию, которая находит значение производной результата по ядру свёртки.](https://stepik.org/lesson/261477/step/3?unit=242226)

In [632]:
def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """

    result = []
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    for k_out in kernel:
        cont = np.zeros((kernel_size, in_channels))
        for offset in range(len(x)):
            if offset + (kernel_size - 1) < len(x):
                cont += x[offset : offset + kernel_size]

        result.append(cont)

    return np.array(result)

### [Напишите функцию, которая находит значение производной результата по входу.](https://stepik.org/lesson/261477/step/4?unit=242226)

In [629]:
def calculate_conv_x_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """

    result = np.zeros(x.shape)
    der = 0
    x_cont = None

    for i, v in np.ndenumerate(x):
        x_cont = x.copy()
        x_cont[i] = x_cont[i] + 1
        result[i] = (
            apply_convolution(x_cont, kernel, bias).sum()
            - apply_convolution(x, kernel, bias).sum()
        )

    return result

### [Напишите функцию для расчёта ширины рецептивного поля.](https://stepik.org/lesson/261477/step/5?unit=242226)

In [686]:
def calculate_receptive_field(layers):
    """
    layers - list of LayerInfo

    returns int - receptive field size
    """
    res = 1

    for layer in layers:
        res += (layer.kernel_size - 1) * layer.dilation

    return res